In [5]:
#feature extraction from previous segmented images 
#to extract features for each image in the dataset such as texture, color, shape, size, etc
#and store them in a file
import cv2
import os
import numpy as np
from skimage.feature import hog
from skimage import data, exposure
import pywt
from imutils import face_utils
import numpy as np
from skimage import io, feature, measure
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Feature Extractor using wavelets
class feature_extract:
    def __init__(self, wavelet= 'haar', level=1):
        self.wavelet = wavelet
        self.level = level

    def  extract(self, image):
        #convert image to float and scale to [0,1]
        image = image.astype('float32')/255

        #apply wavelet transform
        coeffs = pywt.wavedec2(image, self.wavelet, level=self.level)
        #extract features
        features = []
        for i, coeffs in enumerate(coeffs):
            if i == 0:
                #approximation coefficients
                features += self._extract_coeff_features(coeffs, 'A')
            
            else:
                #deatil coeeficients
                features += self._extract_coeff_features(coeffs[0], 'H')
                features += self._extract_coeff_features(coeffs[1], 'V')
                features += self._extract_coeff_features(coeffs[2], 'D')

        return features
    
    def _extract_coeff_features(self, coeff, label):
        #calculate statistics for wavelet coefficients
        features = []
        features.append(np.mean(coeff))
        features.append(np.std(coeff))
        features.append(np.median(coeff))
        features.append(np.max(coeff))
        features.append(np.min(coeff))

        # Add label to feature names
        features = [f'{label}_{f}' for f in features]

        return features
    

In [ ]:
#feature extraction using shape based

class shape_features:
    def __init__(self, image):
        self.image = image

    def extract_features(self, image_path):
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # detect and extract features
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
        faces = detector(gray)
        for face in faces:
            shape = predictor(gray, face)
            shape = face_utils.shape_to_np(shape)
            # loop over the (x, y)-coordinates for the facial landmarks
            # and draw them on the image
            for (x, y) in shape:
                cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
        # show the output image with the face detections + facial landmarks
        cv2.imshow("Output", image)
        cv2.waitKey(0)

In [7]:
# feature extraction using machine learning models -  Random forest classifier
#load the input data


input_data  = "D:\master_thesis\datasets\segmented\glioma"

#lists to store the extracted features and lables
feature = []
labels =[]

#extract the features from each iamge in the dataset
for image_path, label in input_data:
    image = io.imread(image_path, as_gray=True)
    labels.append(label)

    #shape features
    contours = measure.find_contours(image, 0.5)
    num_countours = len(contours)
    perimeter = np.sum([measure.perimeter(c) for c in contours])

    #texture features 
    lbp = feature.local_binary_pattern(image, P=8, R=1)
    histogram = np.histogram(lbp, bins=256, range=(0,256))[0]
    texture_features = histogram/np.sim(histogram)

    #intensity features
    mean_intensity = np.mean(image)
    std_intensity = np.std(image)

    #combine the extracted features into feature vector
    feature_vector = [num_countours,perimeter] + texture_features.tolist()+ [mean_intensity,std_intensity]

    #append the feature vector and label to repective labels
    feature.append(feature_vector)
    labels.append(label)

#convert the features and label lists to numpy arrays
X = np.array(feature)
Y = np.array(labels)

#split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

#train a machine learning model
model = RandomForestClassifier()
model.fit(X_train,Y_train)

#make predictions on the test set
y_pred = model

#evaluate the model
accuray = accuracy_score(Y_test, y_pred)
print("Accuracy: ",accuray)



ValueError: not enough values to unpack (expected 2, got 1)